![encodelogo](images/encodelogo.gif)

# Exploration of ENCODE cell types

This notebook will explore ENCODE [CTCF](https://www.ncbi.nlm.nih.gov/gene/10664) ChIP-seq data across different cell types. We will first narrow down the ENCODE data to explore by querying the [ENCODE Portal](https://www.encodeproject.org) using its [REST API](https://www.encodeproject.org/help/rest-api/). Then, we will process the data into a more manageable format before visualizing cross-cell-type Spearman correlations.

This notebook has been tested using Python 3.7. If you are using the Anaconda distribution, you should only need to install *deepTools* via `pip install deeptools`. 

In [2]:
%matplotlib inline

import deeptools.getScorePerBigWigBin as score_bw
from deeptools.correlation import Correlation
import numpy as np
import matplotlib.pyplot as plt
import os
import pprint
import pandas as pd
import requests
from urllib.parse import urlparse

## Obtain ENCODE metadata
Here is a [link](https://www.encodeproject.org/search/?type=Experiment&status=released&assay_title=ChIP-seq&biosample_ontology.classification=cell+line&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&target.investigated_as=transcription+factor&replicates.library.biosample.treatments%21=%2A&award.project=ENCODE&award.rfa=ENCODE3&target.label=CTCF) to a small search set of ENCODE CTCF ChIP-seq experiments on untreated samples in various cell lines. Using the REST API and the [ENCODE experiment search](https://www.encodeproject.org/search/?type=Experiment) we can obtain the metadata for these experiments programatically, including more information about the cell lines that were used. The *frame=embedded* query parameter allows us to obtain a full expansion of the embedded *biosample_ontology* objects containing biosample metadata of interest. Searches from the website appear in the URL of the browser, which can be copy-pasted into the script below (don't forget to add *frame=embedded* and *format=json*).

In [20]:
url = (
    'https://www.encodeproject.org/search/?type=Experiment'
    '&status=released'
    '&assay_title=ChIP-seq'
    '&biosample_ontology.classification=cell+line'
    '&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens'
    '&target.investigated_as=transcription+factor'
    '&replicates.library.biosample.treatments!=*'
    '&award.project=ENCODE'
    '&award.rfa=ENCODE3'
    '&target.label=CTCF'
    '&frame=embedded'
    '&format=json'
)
r = requests.get(url)
experiments = r.json()['@graph']
# Obtain a subset of the biosample metadata.
biosample_ontologies = []
for e in experiments:
    # Get the name of the cell line, as well as more
    # general categorical information.
    biosample_ontology_condensed = {
        k: v
        for k, v in e.get('biosample_ontology', {}).items()
        if k == 'term_name' or k.endswith('_slims')
    }
    biosample_ontology_condensed.update(
        {'dataset': e.get('@id')}
    )
    biosample_ontologies.append(biosample_ontology_condensed)
biosample_metadata = pd.DataFrame(biosample_ontologies)
biosample_metadata

,cell_slims,dataset,developmental_slims,organ_slims,system_slims,term_name
0,"[epithelial cell, cancer cell]",/experiments/ENCSR265ARE/,"[mesoderm, endoderm, ectoderm]","[epithelium, prostate gland]",[reproductive system],VCaP
1,[epithelial cell],/experiments/ENCSR303GFI/,"[mesoderm, endoderm, ectoderm]","[epithelium, prostate gland]",[reproductive system],RWPE1
2,[cancer cell],/experiments/ENCSR315NAC/,"[mesoderm, endoderm, ectoderm]",[prostate gland],[reproductive system],LNCAP
3,[cancer cell],/experiments/ENCSR460LGH/,"[mesoderm, endoderm, ectoderm]",[prostate gland],[reproductive system],C4-2B
4,[epithelial cell],/experiments/ENCSR856JJB/,"[mesoderm, endoderm, ectoderm]","[epithelium, prostate gland]",[reproductive system],RWPE2
5,[cancer cell],/experiments/ENCSR857PBV/,"[mesoderm, endoderm, ectoderm]",[prostate gland],[reproductive system],22Rv1
6,"[epithelial cell, cancer cell]",/experiments/ENCSR203QEB/,[],"[epithelium, pancreas]",[endocrine system],Panc1
7,"[stem cell, induced pluripotent stem cell]",/experiments/ENCSR932ZMX/,[ectoderm],[skin of body],[integumental system],GM23338
8,"[stem cell, induced pluripotent stem cell]",/experiments/ENCSR987GXT/,[ectoderm],[skin of body],[integumental system],GM23338
9,"[lymphoblast, hematopoietic cell]",/experiments/ENCSR430YRJ/,[mesoderm],[bone element],"[immune system, circulatory system]",KMS-11


The information we have gathered here will hopefully give us some biological intuition as we explore the data. Next, we will use this list of experiments to gather a set of bigWig files that we will use for our analysis, again using the REST API. Here, we are interested in released signal p-value files from pooled replicates processed on GRCh38. We need to pass the query parameter `limit=all` to return the full list of results, otherwise it will be truncated to the first 25 results.

In [37]:
# Parse the experiments into query parameters.
datasets = [
    '&dataset={}'.format(e.get('@id'))
    for e in experiments
]
url = (
    'https://www.encodeproject.org/search/?type=File'
    '&status=released'
    '&output_type=signal p-value'
    '&assembly=GRCh38'
    '&field=dataset'
    '&field=biological_replicates'
    '&field=cloud_metadata.url'
    '&frame=object'
    '&format=json'
    '&limit=all'
    '&{}'.format(''.join(datasets))
)
r = requests.get(url)
# Filter the files to only those belonging to multiple replicates.
files = [
    f
    for f in r.json()['@graph']
    if len(f['biological_replicates']) > 1
]
file_metadata = pd.DataFrame(files)
# Flatten JSON.
file_metadata['cloud_metadata'] = file_metadata.cloud_metadata.apply(lambda x: x.get('url'))
# Rename column.
file_metadata = file_metadata.rename(columns={'@id': 'file'})
# Join with biosample_metadata.
merged_df = file_metadata.merge(biosample_metadata, how='outer', on='dataset')
merged_df = merged_df[[
    'file',
    'dataset',
    'cell_slims',
    'developmental_slims',
    'organ_slims',
    'system_slims',
    'term_name',
    'biological_replicates',
    'cloud_metadata'
]]
merged_df

,file,dataset,cell_slims,developmental_slims,organ_slims,system_slims,term_name,biological_replicates,cloud_metadata
0,/files/ENCFF166JWH/,/experiments/ENCSR240PRQ/,"[epithelial cell, cancer cell]",[endoderm],"[colon, epithelium, intestine, large intestine]",[digestive system],HCT116,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/12...
1,/files/ENCFF345AEO/,/experiments/ENCSR402IDP/,[cancer cell],[mesoderm],"[blood, bodily fluid]",[circulatory system],MM.1S,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/10...
2,/files/ENCFF488ISB/,/experiments/ENCSR084RDK/,"[hematopoietic cell, B cell, leukocyte, cancer...",[],[],[immune system],DOHH2,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/08...
3,/files/ENCFF677UYW/,/experiments/ENCSR072EUE/,[cancer cell],[mesoderm],"[bone marrow, bone element]","[skeletal system, immune system]",OCI-LY1,"[1, 2]",https://encode-public.s3.amazonaws.com/2016/11...
4,/files/ENCFF632AHW/,/experiments/ENCSR987GXT/,"[stem cell, induced pluripotent stem cell]",[ectoderm],[skin of body],[integumental system],GM23338,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/01...
5,/files/ENCFF016FKV/,/experiments/ENCSR359LOD/,[],"[mesoderm, endoderm, ectoderm]",[prostate gland],[reproductive system],PC-3,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/01...
6,/files/ENCFF727WHR/,/experiments/ENCSR203QEB/,"[epithelial cell, cancer cell]",[],"[epithelium, pancreas]",[endocrine system],Panc1,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/07...
7,/files/ENCFF651PRW/,/experiments/ENCSR856JJB/,[epithelial cell],"[mesoderm, endoderm, ectoderm]","[epithelium, prostate gland]",[reproductive system],RWPE2,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/07...
8,/files/ENCFF059EPZ/,/experiments/ENCSR243INX/,[cancer cell],[endoderm],[lung],[respiratory system],PC-9,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/01...
9,/files/ENCFF769KKD/,/experiments/ENCSR560BUE/,"[epithelial cell, cancer cell]",[ectoderm],"[epithelium, mammary gland]",[integumental system],MCF-7,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/03...


## Download files
Note that in the `cloud_metadata.url` field is a direct HTTPS link to the file object in *s3://encode-public*. Here we will use *Wget*, but it is also possible to obtain the files via `aws s3 cp` or by using `boto3` in Python. As a cloud-alternative to downloading the files see [how to mount *s3://encode-public* on an EC2 instance](https://github.com/ENCODE-DCC/encode-data-usage-examples/blob/master/mount_s3_bucket_and_run_jupyter_on_ec2.ipynb).

In [24]:
# Uncomment to download files to data/.
#os.system('cd data/')
#for url in merged_df.cloud_metadata.values:
#    os.system(wget {}'.format(url))

## Analyze the data
Now that we have the files, we can compute correlations across all of the datasets, which in this case will result in a 24 by 24 symmetric matric. Using the *deepTools* library, we will first compute average scores across 10kb regions, then compute and plot correlations on the averaged data. This workflow has been adapted from the *deepTools* *plotCorrelation* and *multiBigwigSummary* Python scripts using components of the [deepTools API](https://deeptools.readthedocs.io/en/develop/content/api.html). There is nothing significant about the 10kb regions (try changing this parameter and see what happens).

In [41]:
def make_corr_plot(
        bw_files,
        chrs_to_skip,
        bin_size=10000,
        method='pearson',
        fileset_name='result',
        blacklist=None,
        labels=None,
        output_dir=''
    ):
    """ 
    A two-step workflow approximating the sequential usage of deepTools
    command line tools multiBigWigSummary and plotCorrelation. The bigWig
    files are binned (10kb regions by default) and saved as .npz files
    before correlation matrices are computed and plots saved as png images.
    """
    assert method in ('pearson', 'spearman'), 'Invalid correlation method'
    # Autogenerate labels from filename if not provided
    if not labels:
        labels = [filename.split('/')[-1].split('.')[0] for filename in bw_files]
    # Generate a name for the unique combination
    test_name = fileset_name + '_' + method
    if blacklist:
        blacklist_title = 'Blacklisted '
        test_name += '_blacklisted'
    else:
        blacklist_title = ''
    image_name = test_name + '.png'
    # Bin the bigwig data in 10kb increments
    num_reads_per_bin = score_bw.getScorePerBin(
        bw_files,
        bin_size,
        chrsToSkip=chrs_to_skip,
        blackListFileName=blacklist
    )
    # Write to npz file
    filename = output_dir + test_name + '.npz'
    with open(filename, "wb") as f:
        np.savez_compressed(f, matrix=num_reads_per_bin, labels=labels)
    # Compute the correlations
    corr = Correlation(filename, method, labels=labels)
    plot_title = '{}{} Correlation of {}'.format(blacklist_title, method.capitalize(), fileset_name)
    # Create a png file of correlation heatmap
    image_path = output_dir + image_name
    corr.plot_correlation(image_path, plot_title=plot_title)
    return image_path

Now we need to prepare arguments to this function. We will compute correlations only on chromosome 1 (chr1). In order to do so, we need to pass a list of chromosomes to skip. We can use the [GRCh38 chrom.sizes file](https://www.encodeproject.org/files/GRCh38_EBV.chrom.sizes/), to find the names of chromosomes and scaffolds that we would like to skip. It is available on S3 [here](https://encode-public.s3.amazonaws.com/2016/01/06/89effdbe-9e3f-48c6-9781-81e565ac45a3/GRCh38_EBV.chrom.sizes.tsv). 

In [26]:
%%bash
# cd data && wget https://encode-public.s3.amazonaws.com/2016/01/06/89effdbe-9e3f-48c6-9781-81e565ac45a3/GRCh38_EBV.chrom.sizes.tsv
ls

ENCFF004TQX.bigWig
ENCFF016FKV.bigWig
ENCFF019UDG.bigWig
ENCFF059EPZ.bigWig
ENCFF111QNV.bigWig
ENCFF166JWH.bigWig
ENCFF289MVP.bigWig
ENCFF318ASK.bigWig
ENCFF325GYQ.bigWig
ENCFF345AEO.bigWig
ENCFF436WYT.bigWig
ENCFF488ISB.bigWig
ENCFF494OPQ.bigWig
ENCFF539QXW.bigWig
ENCFF543QNU.bigWig
ENCFF632AHW.bigWig
ENCFF651PRW.bigWig
ENCFF677UYW.bigWig
ENCFF727WHR.bigWig
ENCFF733AOC.bigWig
ENCFF769KKD.bigWig
ENCFF799SEU.bigWig
ENCFF831DQC.bigWig
ENCFF915YAY.bigWig
GRCh38_EBV.chrom.sizes.tsv


--2019-04-08 21:47:12--  https://encode-public.s3.amazonaws.com/2016/01/06/89effdbe-9e3f-48c6-9781-81e565ac45a3/GRCh38_EBV.chrom.sizes.tsv
Resolving encode-public.s3.amazonaws.com (encode-public.s3.amazonaws.com)... 52.218.232.195
Connecting to encode-public.s3.amazonaws.com (encode-public.s3.amazonaws.com)|52.218.232.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11686 (11K) [binary/octet-stream]
Saving to: ‘GRCh38_EBV.chrom.sizes.tsv’

     0K .......... .                                          100%  193M=0s

2019-04-08 21:47:12 (193 MB/s) - ‘GRCh38_EBV.chrom.sizes.tsv’ saved [11686/11686]



In [31]:
chrom_sizes = pd.read_csv('data/GRCh38_EBV.chrom.sizes.tsv', sep='\t', header=None)
chrom_sizes

,0,1
0,chr1,248956422
1,chr2,242193529
2,chr3,198295559
3,chr4,190214555
4,chr5,181538259
5,chr6,170805979
6,chr7,159345973
7,chr8,145138636
8,chr9,138394717
9,chr10,133797422


We can see the huge number of scaffolds that are don't pertain to a particular chromosome. We will leave these out of our analysis by adding them to the list of chromosomes to skip.

In [ ]:
data_dir = 'data/'
# Create set of all chromosomes in GRCh38
chr_labels = set(chrom_sizes[0].tolist())
# Create set with desired label
desired_chrs = {'chr1'}
# Using set difference create a list of chromosomes to skip
chrs_to_skip = list(chr_labels - desired_chrs)
# Use the Spearman correlation for comparison, it is less susceptible to outliers than Pearson correlation
method = 'spearman'
# Generate the local bigWig file paths and cell line labels
bw_files = []
bw_labels = []
for f in merged_df.to_dict('record')
    bw_file = data_dir + os.path.basename(f.get('cloud_metadata'))
    bw_label = f.get('term_name')
    bw_files.append(bw_file)
    bw_labels.append(bw_label)
imagename = make_corr_plot(
    bw_files,
    chrs_to_skip,
    method=method,
    fileset_name='CTCF_ChIP-seq',
    labels=bw_labels,
    output_dir='output_images/'
)
print(imagename)

## Plot the results
Now that we have our correlation plot, let's display it. Conveniently, the *deepTools* *plot_correlation* method plots hierarchical clustering results along one end of the matrix. 

In [45]:
fig, ax = plt.subplots(1, 1)
scale = 40
img = plt.imread(imagename)
imgplot = ax.imshow(img)
fig.set_size_inches(tuple([scale*i for i in fig.get_size_inches()]))

The most correlated cell types here are RWPE1, C4-2B, and LNCAP. What do these cell lines have in common?

In [46]:
interesting_lines = ['RWPE1', 'C4-2B', 'LNCAP']
merged_df[merged_df.term_name.isin(interesting_lines)]

,file,dataset,cell_slims,developmental_slims,organ_slims,system_slims,term_name,biological_replicates,cloud_metadata
14,/files/ENCFF289MVP/,/experiments/ENCSR460LGH/,[cancer cell],"[mesoderm, endoderm, ectoderm]",[prostate gland],[reproductive system],C4-2B,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/07...
15,/files/ENCFF494OPQ/,/experiments/ENCSR303GFI/,[epithelial cell],"[mesoderm, endoderm, ectoderm]","[epithelium, prostate gland]",[reproductive system],RWPE1,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/07...
23,/files/ENCFF325GYQ/,/experiments/ENCSR315NAC/,[cancer cell],"[mesoderm, endoderm, ectoderm]",[prostate gland],[reproductive system],LNCAP,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/07...


Cool! These all appear to be prostate cell lines, which could explain their high correlation. Interestingly though, other prostate cell lines, like 22rv1 and RWPE2 don't appear as part of this cluster. We also see a broad and loosely defined cluster around *Loucy* (darker red than other clusters). Let's look at a few cell lines there as well:

In [47]:
interesting_lines = ['Loucy', 'SU-DHL-6', 'A673']
merged_df[merged_df.term_name.isin(interesting_lines)]

,file,dataset,cell_slims,developmental_slims,organ_slims,system_slims,term_name,biological_replicates,cloud_metadata
13,/files/ENCFF799SEU/,/experiments/ENCSR464DKE/,"[hematopoietic cell, T cell, leukocyte, cancer...",[mesoderm],"[blood, bodily fluid]","[immune system, circulatory system]",Loucy,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/03...
16,/files/ENCFF318ASK/,/experiments/ENCSR125DKL/,[cancer cell],[],[],[],SU-DHL-6,"[1, 2]",https://encode-public.s3.amazonaws.com/2018/03...
18,/files/ENCFF915YAY/,/experiments/ENCSR611JJS/,[cancer cell],[],[musculature of body],[musculature of body],A673,"[1, 2]",https://encode-public.s3.amazonaws.com/2017/10...


## Conclusions
We can see that these are all cancer cells. Here are some exercises left to the reader:

1. How would this analysis look on a different chromosome? Does it change meaningfully?

2. How would this analysis look if we used ENCODE ChIP-seq data for a different transcription factor? Would we expect similar clusters to form?  What about a histone mark like H3K4me3?

3. Did the results here match our expectations for CTCF?